# Suncal API User Manual: Reverse Uncertainty Propagation Calculations

The Reverse Uncertainty calculation answers the question "How much uncertainty is accepatble in one input variable to acheive some required uncertainty in the output of the model?"

Start by importing `reverse` from suncal.

In [1]:
import suncal
from suncal import reverse

Consider the example of measuring density of a cylinder by measuring it's height, diameter, and mass:

$\rho = \frac{m}{\pi (d/2)^2 h}$

The product specification for a cylinder lists a density requirement of $2.00 \pm 0.06 \, \mathrm{g/cm^3}$ (95% confidence). From the manufacturing process, the height and diameter are given as $h=1.5$ cm and $d=1.0$ cm with uncertainty of $0.01$ cm (95% confidence). What range of measured mass values result in an acceptable density?

To properly set up this calculation, use `suncal.reverse.Modelreverse`.
It works similarly to `suncal.Model`, taking one or more model functions as parameters. However, the `ModelReverse` must also specify the nominal target value, target uncertainty, and variable to solve for.

Here the cylinder density example is set up:

In [2]:
# Define the cylinder parameters
rho_req = 2
urho_req = .06     # 95%
d, h = 1.0, 1.5
ud, uh = .01, .01  # 95%

In [3]:
model = reverse.ModelReverse('rho = m / (pi/4 * d**2 * h)',
                          targetnom=rho_req,
                          targetunc=urho_req/1.96,
                          solvefor='m')
model.var('h').measure(h).typeb(unc=uh, conf=.95)
model.var('d').measure(d).typeb(unc=ud, conf=.95)
# Don't need to specify 'm' as it will be solved for.

# Reverse calc value of m to acheive rho +/- urho
result = model.calculate()
result

## GUM reverse uncertainty

$\rho = \frac{4 m}{\pi d^{2} h}$

 Combined uncertainty:

$u_{\rho} = \frac{4 \sqrt{\frac{d^{2} h^{2} u_{m}^{2} + d^{2} m^{2} u_{h}^{2} + 4 h^{2} m^{2} u_{d}^{2}}{d^{6} h^{4}}}}{\pi}$

solved for uncertainty of input:

$u_{m} = \frac{\sqrt{\pi^{2} d^{6} h^{4} u_{\rho}^{2} - \pi^{2} d^{6} h^{2} \rho^{2} u_{h}^{2} - 4 \pi^{2} d^{4} h^{4} \rho^{2} u_{d}^{2}}}{4 d h}$

 For output value of 
2.000
 ± 
0.031
 (k=1),

required input value is 
2.356
 ± 
0.026
 (k=1).



---

## Monte Carlo reverse uncertainty

For output value of 
2.000
 ± 
0.031
 (k=1), required input value is: 
2.356
 ± 
0.026
 (k=1).

From this result, the uncertainty in mass must be 0.026 or smaller to achieve the desired uncertainty in density.

The result has both a `gum` and `montecarlo` attribute, which contain the solve-for value, uncertainty in the solve-for value, and other parameters.

In [4]:
f'{result.gum.funcname} = {result.gum.solvefor_value:.3f} ± {result.gum.u_solvefor_value:.3f}'

'rho = 2.356 ± 0.026'

## Understanding the Reverse Calculation

A naive solution to the reverse problem be to simply flip around the measurement function, solving for $m$, and run through the calculator as such:

In [5]:
model = suncal.Model('m = rho * pi/4 * d**2 * h')
model.var('rho').measure(rho_req).typeb(unc=urho_req, conf=.95)
model.var('h').measure(h).typeb(unc=uh, conf=.95)
model.var('d').measure(d).typeb(unc=ud, conf=.95)
result_check = model.calculate_gum()
result_check

|Function   | Nominal   | Std. Uncertainty  |
|----------|---------|------------------|
|$m$   | 2.356  | 0.044          |

However, checking this computed mass, $2.36 \pm 0.044$ g, using a forward calculation for density produces an incorrect result! We get 95% coverage range of $\pm 0.085$ instead of $\pm 0.060$ as required. In other words, using this mass limit, there is a greater than the acceptable 5% chance of accepting an out-of-tolerance part.

The incorrect result stems from two factors: (1) the linearity of the Taylor expansion used by GUM method may not be reversible, (2) the calculation neglects the fact that $\rho$ is now correlated with the other input variables.

In [6]:
# Check the naive answer
expected = result_check.expected['m']
uncert = result_check.uncertainty['m']

model = suncal.Model('rho = m / (pi/4 * d**2 * h)')
model.var('m').measure(expected).typeb(std=uncert)
model.var('h').measure(h).typeb(unc=uh, conf=.95)
model.var('d').measure(d).typeb(unc=ud, conf=.95)
result_check2 = model.calculate_gum()
result_check2.expand('rho', .95)

0.04100125901604559

A proper reverse uncertainty calculation can be performed by finding $u_{\rho}$ and solving for $u_m$. Suncal can do this for you, along with a correct Monte Carlo approach which includes correlation effects, using `suncal.reverse.ModelReverse`, as run above.

Taking the `ModelReverse` result run above ($2.36 \pm 0.026$ g) back in, we should get the required 95% uncertainty of $2.0 \pm 0.060$ g.

In [7]:
expected = result.gum.solvefor_value
uncert = result.gum.u_solvefor_value

model = suncal.Model('rho = m / (pi/4 * d**2 * h)')   # Set up in forward direction
model.var('m').measure(expected).typeb(std=uncert)
model.var('h').measure(h).typeb(unc=uh, conf=.95)
model.var('d').measure(d).typeb(unc=ud, conf=.95)
result_check = model.calculate_gum()
result_check.expand('rho', conf=0.95)

0.05999889755854056

### Using worst-case values

Another common, yet incorrect, approach is to simply calculate the worst case value for mass using the highest and lowest values for each input variable. This method also results in a 95% density uncertainty well above the $0.0600$ requirement as shown here.

In [8]:
# Calculate highest and lowest 95% range input values
rho_hi = rho_req + urho_req
rho_lo = rho_req - urho_req
d_hi = d + ud
d_lo = d - ud
h_hi = h + uh
h_lo = h - uh

# Highest and lowest mass
mhi = rho_hi * 3.1415/4 * d_hi**2 * h_hi
mlo = rho_lo * 3.1415/4 * d_lo**2 * h_lo
m = (mhi + mlo)/2
um = (mhi - m)
print('Worst Case m = {:.6f} +/- {:.6f} (95%)'.format(m, um))

# Plug in to forward calculation, see resulting rho range
model = suncal.Model('rho = m / (pi/4 * d**2 * h)')
model.var('m').measure(m).typeb(unc=um, conf=.95)
model.var('h').measure(h).typeb(unc=uh, conf=.95)
model.var('d').measure(d).typeb(unc=ud, conf=.95)
model.calculate()

Worst Case m = 2.358560 +/- 0.133532 (95%)


|Function   | Method       | Nominal   | Std. Uncertainty   | 95% Coverage   | k         | Deg. Freedom  |
|----------|------------|---------|------------------|--------------|---------|--------------|
|$\rho$   | GUM          | 2.002  | 0.062           | ± 0.12     | 1.960     | inf           |
|$\rho$   | Monte Carlo  | 2.002  | 0.062           | (1.882, 2.124) | 1.960     | -             |